# Lab 7 - Web Scraping
---
In today's lab, we are going to download data from the internet using an API. API stands for application programming interface. Companies often create APIs as a way to allow users to more directly interact with their servers to retrieve data. Today, we are going to be using CKAN's API to download data from the City of Toronto's Open Data Portal to get some experience working with larger datasets.

In [ ]:
# Run this cell to set up your notebook
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import warnings
import requests
from ckan_utils import *
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

# Ensure that Pandas shows at least 280 characters in columns, so we can see full tweets
pd.set_option('max_colwidth', 280)

%matplotlib inline
import re
import json

## Setup
---
If you are having trouble with any of the following cells, you can read in an already downloaded version of the dataset. 

In [ ]:
# toronto public library info

# Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = { "id": "library-branch-general-information"}
package = requests.get(url, params = params).json()

In [ ]:
package

This is an example of another `python` data structure called a *dictionary*. Dictionaries store *values* by associating them with a *key* rather than by an integer index. You can access the values stored in a dictionary using bracket notation just like a list. For example:

In [ ]:
# In this dictionary, the keys are strings, and the values are all numbers
d = {'a': 1,
    'b': 2,
    'c': 3}

d['a']

In the case of `package`, it is an example of a nested dictionary. To access its values, we need to use a key of a key. It would also appear that there are many values to access, so instead of hard coding the keys one at a time, let's use Python's list comprehension to print all of them out at once. 

In [ ]:
package.keys()

In [ ]:
package.values()

In [ ]:
# print the metadata
[x for x in package["result"]["resources"]]

That's a lot of information compared to what we want- a simple dataset of library information! There are some important fields here to take note of that will guide how you download the information through the API. Note that the first resource has `datastore_active == True`. This means an instance of the data is stored on the Open Data portal's database. Not all records will have this value as `True`, as you can see in the event that a resource can be downloaded in `csv`, `json`, or `xml` format. For now, we will download the instance where this is true, but later in the lab we will learn what to do when the data is stored elsewhere. 

In [ ]:
# To get resource data:
# iterate over the resources
for idx, resource in enumerate(package["result"]["resources"]):

    # set a condition for when you want to access the resource:
    if resource["datastore_active"]:

        # to get all records in CSV format (this is specific to CKAN's API)
        url = base_url + "/datastore/dump/" + resource["id"]
        # do a GET request on the url and access its text attribute
        resource_dump_data = requests.get(url).text
        # read the raw csv text into a pandas dataframe to work with it
        tpl_libraries = pd.read_csv(StringIO(resource_dump_data), sep=",")
tpl_libraries.head()

In [ ]:
tpl_libraries.columns

Now that we have information on the libraries, let's see if we can find out a little more about them using the dataset `library-branch-programs-and-events-feed`. 

In [ ]:
# Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To interact with the API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = { "id": "library-branch-programs-and-events-feed"}
package = requests.get(url, params = params).json()

In [ ]:
package

In [ ]:
[x for x in package["result"]["resources"]]

In [ ]:
# To get resource data:
# iterate over the resources
for idx, resource in enumerate(package["result"]["resources"]):

    # set a condition for when you want to access the resource:
    if resource["datastore_active"]:

        # to get all records in CSV format (this is specific to CKAN's API)
        url = base_url + "/datastore/dump/" + resource["id"]
        # do a GET request on the url and access its text attribute
        resource_dump_data = requests.get(url).text
        # read the raw csv text into a pandas dataframe to work with it
        tpl_events = pd.read_csv(StringIO(resource_dump_data), sep=",")
tpl_events.head()

## Data Cleaning
---
Now, we want to extract out only the columns that are relevant to us. Discarding columns that do not help us answer our question can be helpful because it prevents the computer from having to do unnecessary computations. However, if we want to be able to connect any conclusions we make after we get rid of columns, it is helpful to keep an identifying column in your `DataFrame` even if you are not performing analyses on it.

You can read about all of the columns under the data features tab [here](https://open.toronto.ca/dataset/library-branch-programs-and-events-feed/). It's good practice to read as much as you can about the metadata of a dataset, when and where it is available to minimize the amount of guesswork or reconstruction you'll have to do. 

In [ ]:
tpl_events.columns

In [ ]:
tpl_events = tpl_events[['_id', 'title', 'startdate', 'enddate', 'starttime', 'endtime',
       'length', 'library',  'description',  'id',
       'rcid', 'eventtype1', 'eventtype2', 'eventtype3', 'agegroup1',
       'agegroup2', 'agegroup3',  'lastupdated']]
tpl_events.head()

## Reshaping and pivoting dataframes

But that's not all we can do to the data to make it easier to work with. It would be nice if the event type and age group columns were pivoted to one rather than three separate columns each. We can reshape dataframes into a 'long' format using the `melt` function. 

There is an important distinction to make in pandas datatypes. Normally, `None` is not a string in Python, it has a particular value which you can think of as null. But columns in Pandas must all be of a single type, and when a `None` is in a column with other strings, it too becomes a string. Therefore, to drop rows with `None`, you must use `!= "None"` rather than `!= None`. 

In [ ]:
event_types = tpl_events.melt(id_vars = ["id", "library"], value_vars = ["eventtype1", "eventtype2", "eventtype3"], value_name = "eventtype")
event_types = event_types[event_types['eventtype'] != "None"].drop(columns = "variable")
event_types

In [ ]:
age_groups = tpl_events.melt(id_vars = ["id", "library"], value_vars = ["agegroup1", "agegroup2", "agegroup3"], value_name = "agegroup")
age_groups = age_groups[age_groups['agegroup'] != "None"].drop(columns = "variable")
age_groups

In [ ]:
# join these back to tpl_events
tpl_events_long = tpl_events.drop(columns = ["eventtype1", "eventtype2", "eventtype3", "agegroup1", "agegroup2", "agegroup3"]).merge(event_types, on = ["id", "library"], how = "left").merge(age_groups, on = ["id", "library"], how = "left")
tpl_events_long.head()

Let's use the `.groupby()` method to summarize event types.

The `.groupby()` method takes in a table, a column, and optionally, an aggregate function (the default is count() which counts how many rows have the same value for the column we are grouping by. Other options include sum() and max() or min()). Groupby goes through each row, looks at the column that has been given to it of the current row, and groups each row based on if they have the same value at given column. After it has a list of rows for each distinct column value, it applies the aggregate function for each list, and returns a table of each distinct column value with the aggregate function applied to the rows that corresponded with the column.

Let's see if we can find the most popular library event type.  

In [ ]:
tpl_events_long.groupby('eventtype').size().sort_values(ascending = False)

**Your turn:** Let's find the most common public library event type by age group for each branch. We've provided some starter code, but you need to fill in wherever you see a `...`!

In [ ]:
tpl_events_long.groupby(['...', '...'])['eventtype'].agg(pd.Series.mode)

## Temporal Data
---
Another facet of urban data that you may want to analyze is the time at which something occurs. `python` compares strings by assigning values to the letters themselves based on their position in the alphabet. We want to convert these strings to `datetime` objects, which will tell `python` at what time the precipitation was measured.

Notice that we are not adding parentheses at the end of each line. That is because the `.day` and `.month` are not *functions* we are calling, but rather *attributes* of the particular `datetime` object. If we want to look at the day of the month library events start on, we can extract these attributes.

In [ ]:
start_date = tpl_events[['id', 'library', 'startdate']].drop_duplicates()
start_date['time'] = pd.to_datetime(start_date['startdate'])
start_date['day'] = start_date['time'].dt.day
start_date['month'] = start_date['time'].dt.month

In [ ]:
start_date['day'].hist();
plt.xlabel("Day of Month")
plt.ylabel("Number of Events");

**Question:** What observations or trends do you notice about this graph?

**Question:** What could be improved about this graph or the process we used to obtain the data that generated it?

## Sentiment Analysis
---
We can use the words the tweets to measure the sentiment, or the positive/negative feeling generated by the description text. To do so we will be using the [VADER (Valence Aware Dictionary and sEntiment Reasoner)](https://github.com/cjhutto/vaderSentiment), which is a rule-based sentiment analysis tool specifically designed for social media. It even includes emojis! Run the following cell to load in the lexicon.

In [ ]:
vader = load_vader()
vader.iloc[500:510, :]

The more positive the polarity of a word, the more positive feeling the word evokes in the reader. All of the words in `vader` are all lowercase, while many of our tweets are not. We need to modify the text in the tweets so that the words in our tweets will match up with the words stored in `vader`. Additionally, we need to remove punctuation since that will cause the words to not match up as well. We will put these modified tweets into another column in our `DataFrame` so that we can still have access to them later.

In [ ]:
# Select our columns of interest
# polarity by age / event type?
descriptions = tpl_events_long[['id', 'library', 'description']].drop_duplicates()

# Set the index of the DataFrame to the tweet ID. This step is necessary
# in order to use our utility functions.
descriptions.set_index('id', inplace = True)

# Remove punctuation and lowercase tweets
descriptions['cleaned'] = clean_string(descriptions['description'])

descriptions.head()

Next, we want to merge our sentiment lexicon with our cleaned tweets. 

In [ ]:
descriptions['polarity'] = compose_polarity(descriptions, vader)
descriptions.head()

Next, we want to see if more polarizing tweets are retweeted more often. To do this, we can plot the `polarity` and `retweet_count` columns against each other.

In [ ]:
tweets_and_retweets.plot('polarity', 'retweet_count', kind='scatter');

**Question:** What conclusions can you draw about polarity and retweets from this graph? How does this compare with your assumptions?

## Your turn!
---
If time allows, try these exercises on your own or as a class!

**Exercise 1:** Using the `gentrification_oak` tweets, make a histogram of the time of day the tweets were posted. Note that if you would like the x-axis of the plot to reflect the correct time of day, you will have to convert the time from UTC to PDT.

In [ ]:
# YOUR CODE HERE

**Exercise 2:** Try scraping tweets from multiple locations and the same hashtag. Make a histogram for each location and see if there are any differences in the distribution of polarity of the tweets. Feel free to use multiple cells to avoid querying the API repeatedly.

In [ ]:
# YOUR CODE HERE